In [ ]:
from python_simulation_manager.cpp_handler import CppExperiment, CppExperimentBuilder
from python_simulation_manager.output import ExperimentOutput
from typing import override
from pathlib import Path
import numpy as np

In [ ]:
class MyOutput(ExperimentOutput):
    def __init__(self, out_path):
        super().__init__(out_path)
        self.title   = None
        self.result1 = []
        self.result2 = []

    @override
    def parse_output(self, line_number, line):
        if line_number == 0:
            self.title = line.strip()
            return
        
        slines = line.split("=>")
        if line_number == 1:
            self.result1.append(float(slines[1]))
        else:
            self.result2.append(float(slines[1]))

In [ ]:
a, b = "haha,hehe".split(",")
a

In [ ]:
class CppExperimentFactory:
    
    def __init__(self, results_dir: str, exp_name: str):
        self.proj_dir = Path.cwd().parent
        self.builder  = CppExperimentBuilder(self.proj_dir, results_dir, exp_name, verbose_log=False)

    def create_experiment(self, monte_carlo_trials: dict[int, int], temperature: np.ndarray) -> CppExperiment:
        executable  = self.proj_dir / "c_example" / "build" / "c_example.exe"

        self.builder.set_output_type(MyOutput)
        self.builder.add_static_parameter("temperature", temperature)
        self.builder.add_scaling_parameter("monte_carlo_trials", monte_carlo_trials)
        self.builder.set_scale_variable_names(["length"])        
        self.builder.set_executable(executable)
        experiment = self.builder.build()
        return experiment
    
    def load(self, lengths: list[int]) -> CppExperiment:
        executable  = self.proj_dir / "c_example" / "build" / "c_example.exe"

        self.builder.set_output_type(MyOutput)
        self.builder.set_scale_variables(lengths)
        self.builder.set_scale_variable_names(["length"])        
        self.builder.set_executable(executable)
        experiment = self.builder.build(load_only=True)
        return experiment

In [ ]:
result_folder = "results"
exp_folder    = "c_experiment"

factory = CppExperimentFactory(result_folder, exp_folder)

monte_carlo_trials = {
    2: 1_000,
    3: 5_000,
    4: 10_000
}
temperature = np.arange(0, 10, 1, np.int32)
experiment  = factory.create_experiment(monte_carlo_trials, temperature)
experiment.write_parameter_files()

In [ ]:
experiment.are_parameter_files_available()

Helper function to give meaning to "scale_variables":

In [ ]:
def lengths(exp: CppExperiment) -> list[int]:
    return exp.get_scale_variables()

In [ ]:
for length in lengths(experiment):
    experiment.run_executable(length, verbose=False)

In [ ]:
experiment.are_results_available()

Helper function for type hinting:

In [ ]:
def results(exp: CppExperiment) -> dict[int, MyOutput]:
    return exp.get_results()

In [ ]:
for (length, res) in results(experiment).items():
    
    print(f"For length = {length}:")
    print(f"> title: \"{res.title}\"")
    print(f"> result1 = {res.result1}")
    print(f"> result2 = {res.result2}")
    print("")
    
    assert type(res.result1) == np.ndarray, "Lists are cast to np array automatically!"
    assert type(res.result2) == np.ndarray, "Lists are cast to np array automatically!"

From file

In [ ]:
result_folder = "test_folder"
exp_folder    = "c_experiment"
lengths       = [2,3,4]

exp_loader     = CppExperimentFactory(result_folder, exp_folder)
exp_from_file  = exp_loader.load(lengths) 

In [ ]:
exp_from_file.are_results_available()

In [ ]:
for (length, res) in results(exp_from_file).items():
    
    print(f"For length = {length}:")
    print(f"> title: \"{res.title}\"")
    print(f"> result1 = {res.result1}")
    print(f"> result2 = {res.result2}")
    print("")